In [1]:
!pip install ratsnlp

     |████████████████████████████████| 42 kB 386 kB/s 
     |████████████████████████████████| 806 kB 5.3 MB/s 
     |████████████████████████████████| 398 kB 43.3 MB/s 
     |████████████████████████████████| 2.8 MB 40.4 MB/s 
     |████████████████████████████████| 57 kB 5.4 MB/s 
     |████████████████████████████████| 8.0 MB 9.7 MB/s 
     |████████████████████████████████| 829 kB 43.5 MB/s 
     |████████████████████████████████| 136 kB 39.0 MB/s 
     |████████████████████████████████| 636 kB 38.4 MB/s 
     |██████████████████████████████▎ | 834.1 MB 1.2 MB/s eta 0:00:41tcmalloc: large alloc 1147494400 bytes == 0x3acdc000 @  0x7fcb39321615 0x592b76 0x4df71e 0x59afff 0x515655 0x549576 0x593fce 0x548ae9 0x51566f 0x549576 0x593fce 0x548ae9 0x5127f1 0x598e3b 0x511f68 0x598e3b 0x511f68 0x598e3b 0x511f68 0x4bc98a 0x532e76 0x594b72 0x515600 0x549576 0x593fce 0x548ae9 0x5127f1 0x549576 0x593fce 0x5118f8 0x593dd7
     |████████████████████████████████| 881.9 MB 19 kB/s 
     |██████████

In [2]:
from google.colab import drive
drive.mount('/grive', force_remount=True)

Mounted at /grive


In [5]:
#말뭉치 내려받기
from Korpora import Korpora
nsmc = Korpora.load("nsmc", force_download=True)


    Korpora 는 다른 분들이 연구 목적으로 공유해주신 말뭉치들을
    손쉽게 다운로드, 사용할 수 있는 기능만을 제공합니다.

    말뭉치들을 공유해 주신 분들에게 감사드리며, 각 말뭉치 별 설명과 라이센스를 공유 드립니다.
    해당 말뭉치에 대해 자세히 알고 싶으신 분은 아래의 description 을 참고,
    해당 말뭉치를 연구/상용의 목적으로 이용하실 때에는 아래의 라이센스를 참고해 주시기 바랍니다.

    # Description
    Author : e9t@github
    Repository : https://github.com/e9t/nsmc
    References : www.lucypark.kr/docs/2015-pyconkr/#39

    Naver sentiment movie corpus v1.0
    This is a movie review dataset in the Korean language.
    Reviews were scraped from Naver Movies.

    The dataset construction is based on the method noted in
    [Large movie review dataset][^1] from Maas et al., 2011.

    [^1]: http://ai.stanford.edu/~amaas/data/sentiment/

    # License
    CC0 1.0 Universal (CC0 1.0) Public Domain Dedication
    Details in https://creativecommons.org/publicdomain/zero/1.0/



[nsmc] download ratings_train.txt: 14.6MB [00:00, 66.8MB/s]                            
[nsmc] download ratings_test.txt: 4.90MB [00:00, 32.9MB/s]                           


In [6]:
import os
#nsmc 리뷰를 텍스트형태로 저장
def write_lines(path, lines):
  with open(path, 'w', encoding='utf-8') as f:
    for line in lines:
      f.write(f'{line}\n')

write_lines('/root/train.txt', nsmc.train.get_all_texts())
write_lines('/root/test.txt', nsmc.test.get_all_texts())

In [3]:
#bert directory 만들기
import os
os.makedirs("/gdrive/My Drive/nlpbook/wordpiece", exist_ok = True)

In [7]:
from tokenizers import BertWordPieceTokenizer
wordpiece_tokenizer = BertWordPieceTokenizer(lowercase=False)
wordpiece_tokenizer.train(
    files=["/root/train.txt", "/root/test.txt"],
    vocab_size=10000,
)
wordpiece_tokenizer.save_model("/gdrive/My Drive/nlpbook/wordpiece")

['/gdrive/My Drive/nlpbook/wordpiece/vocab.txt']

In [8]:
# BERT 입력값 예시
from transformers import BertTokenizer
tokenizer_bert = BertTokenizer.from_pretrained(
    "/gdrive/My Drive/nlpbook/wordpiece",
    do_lower_case=False,
)

file /gdrive/My Drive/nlpbook/wordpiece/config.json not found


In [9]:
sentences = [
             "안녕 나는 한강 커피스미스에서 와이파이 연결이 안되는 사람이야",
             "나의 미래는 어디로 흘러가는 걸까",
             "단추 단테 귀여워",
]
tokenized_sentences = [tokenizer_bert.tokenize(sentence) for sentence in sentences]

In [10]:
# 토큰에 붙어있는 '##' 은 해당 토큰이 어절의 시작(띄어쓰기 기준)이 아님을 나타낸다. 
# 예를 들어, '##스' 은 커피스미스 에 붙어있는 토큰이라는 것을 표시(어절 내에서 연속)
tokenized_sentences

[['안녕',
  '나는',
  '한',
  '##강',
  '커피',
  '##스',
  '##미스',
  '##에서',
  '와',
  '##이',
  '##파이',
  '연결',
  '##이',
  '안되는',
  '사람이',
  '##야'],
 ['나의', '미래', '##는', '어디로', '흘러가는', '걸까'],
 ['단', '##추', '단', '##테', '귀여워']]

In [12]:
batch_inputs = tokenizer_bert(
    sentences,
    padding="max_length",
    max_length=12,
    truncation=True,
)

In [13]:
#결과로 input_ids, attention_mask가 만들어지며,
#input_ids의 2, 3은 각각 [CLS], [SEP] 로 문장의 시작과 끝을 말한다.
#attention_mask은 GPT와 동일. 일반 토큰은 1, 패딩 토큰은 0
batch_inputs["input_ids"]

[[2, 8314, 2371, 947, 1454, 8936, 1105, 7799, 1992, 675, 1012, 3],
 [2, 2974, 3660, 1007, 6839, 7105, 7890, 3, 0, 0, 0, 0],
 [2, 275, 1251, 275, 1215, 6041, 3, 0, 0, 0, 0, 0]]

In [14]:
batch_inputs["attention_mask"]

[[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0],
 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]]